In [1]:
import time
import urllib.request
from urllib.request import urlopen
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import random
from urllib.error import HTTPError
from selenium.common.exceptions import NoSuchElementException
import re
import numpy as np
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pymysql
import os
from openpyxl import Workbook
from pyvirtualdisplay import Display

import matplotlib.pyplot as plt
import pandas as pd
import sys
import random

In [2]:
#수집하고 싶은 태그명
tag = 'parasitemovie'

#이미지 저장 폴더 경로 입력
directory = "./parasite/"

In [3]:
options = Options()
# chrome을 전체화면으로 넓히는 옵션입니다.
options.add_argument('--start-fullscreen')

#display = Display(visible=0, size=(1024,768))
#sdisplay.start()
username = "suuu_0829"
password = "rla132489"

collect_driver = webdriver.Chrome(executable_path='./chromedriver')
getdriver = ("https://www.instagram.com/accounts/login/")
collect_driver.get(getdriver)

time.sleep(2)

id_input = collect_driver.find_elements_by_css_selector('#react-root > section > main > div > article > div > div > div > form > div > div > label > input')[0]
id_input.send_keys(username)
password_input = collect_driver.find_elements_by_css_selector('#react-root > section > main > div > article > div > div > div > form > div > div > label > input')[1]
password_input.send_keys(password)

password_input.submit()

time.sleep(3)

chrome_options = Options()

chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")

data = []
pagedown = 0
post_num = 1


collect_driver.get('https://www.instagram.com/explore/tags/'+tag+'/')

time.sleep(3)

body = collect_driver.find_element_by_tag_name("body")

num_of_pagedowns = 3000

while num_of_pagedowns:
    body.send_keys(Keys.END)
    time.sleep(0.2)
    num_of_pagedowns -=1

soup = BeautifulSoup(collect_driver.page_source, 'html.parser')
source = soup.find_all('div',{'class':'v1Nh3 kIKUG _bz0w'})

time = random.randrange(0,3)

for i in range(len(source)):
    file_data = {}
    try:
        collect_driver.implicitly_wait(time)
        #페이지 열기
        collect_driver.get('https://www.instagram.com' + (source[i].find("a")).get('href'))
        #페이지 소스 가져오기
        collect_soup = BeautifulSoup(collect_driver.page_source,'html.parser')
        #게시글 작성자 이름
        user_id = collect_driver.find_element_by_class_name('_6lAjh').text
        file_data['id'] = user_id

        #좋아요 개수
        like = collect_driver.find_element_by_class_name('Nm9Fw').text
        like_count = like.split(' ')[1]
        file_data['like'] = like_count

        #게시글 생성날짜
        post_date = collect_soup.find('time',{'class':'_1o9PC Nzb55'}).get('datetime')  
        file_data['date'] = post_date[0:10]

        #해시태그
        tag_source = collect_soup.find_all('meta',{'property':'instapp:hashtags'})
        tag_list = []
        for j in range(len(tag_source)):
            tag_list.append(tag_source[j].get('content'))
        tag_string = "|".join(tag_list)
        file_data['hashtags'] = tag_string

        #댓글
        comment_list = []
        comment_source = collect_soup.find_all('div',{'class':'C4VMK'})
        for k in range(1,len(comment_source)):
            comment = comment_source[k].find('span').text
            #comment = comment.replace("'",";")
            c_id = comment_source[k].find('a').text
            comment_list.append([comment,c_id])
        file_data['comments'] = comment_list

        #텍스트 본문
        try:
            text_source = collect_soup.find('div',{'class':'C4VMK'}).find('span').text
        except:
            text_source = ""
        finally:
            if str(text_source) == "인증됨":
                text_source = collect_soup.find('div',{'class':'C4VMK'}).find_all('span')[1].text            

        file_data['text'] = text_source

        if post_num % 100 == 0:
            print(post_num)

        post_num +=1

        data.append(file_data)

    except NoSuchElementException:
        print(post_num,"번째 페이지 삭제됨")
        continue
            
    if(post_date[0:10] == '2019-04-07'):
        collect_driver.close()
        
    #scroll down
    #driver.execute_script('arguments[0].scrollIntoView(true);',bottom)
    #pagedown += 1

39
1 번째 페이지 삭제됨
39
1 번째 페이지 삭제됨
39
39
2 번째 페이지 삭제됨
39
39
3 번째 페이지 삭제됨
39
39
39
39
39
39
8 번째 페이지 삭제됨
39
39
39
39
39
39
39
39
39
39
17 번째 페이지 삭제됨
39
39
39
39
39
39
39
39
39
25 번째 페이지 삭제됨
39
39
39
39
39
39
30 번째 페이지 삭제됨
39
39


In [4]:
len(data)

31

In [8]:
user_id = []
like = []
date = []
hashtags = []
comments = []
post = []

# id, like, date, hashtags, comments, text

for i in range(len(data)):
    user_id.append(str(data[i]['id']))
    like.append(str(data[i]['like']))
    date.append(str(data[i]['date']))
    hashtags.append(str(data[i]['hashtags']))
    comments.append(str(data[i]['comments']))
    post.append(str(data[i]['text']))

result = pd.DataFrame({'id':user_id, 'like':like, 'date':date, 'hashtags':hashtags, 'comments':comments, 'post':post})

result.to_csv("instagram.csv", mode='w')

In [ ]:
    '''
    <div id='a'>
      <div>
        <a class='click'>abc</a>
      </div>
    </div>
    '''

In [6]:
data

[{'id': 'thescorpionsoul',
  'like': '3개',
  'date': '2020-03-19',
  'hashtags': '',
  'comments': [['#asur #asurreview #indianwebseries #vootselect #asura #korean_drama #netflixshows #netflix #netflixmovies #netflixandchill #netflixseries #parasitemovie #parasite #netflixoriginal #netflixindia #moviereview #moviereviews #movie #movies #movielover #movielovers #moviebuff #movieaddict #movieworld #movieqoutes #moviescenes #moviescene #moviedate',
    'thescorpionsoul'],
   ['Okay I have to see this now.', '_a_curated_soul']],
  'text': "ASUR.N O N - S P O I L E R - R E V I E WSTORY:- A unique crime thriller that pits two opposing worlds of forensic science & hindu mythological events against each other.LANGUAGE:- Hindi (With English Subtitles)ALERT:- Forget about Sacred Games & Mirzapur, ASUR is India's best web series.RATING:- 5/5POSITIVE:- ASUR is like a drug and it's hangover will make you go crazy. It will leave you in goosebumps. The haunting BGM will scare the shit out of you. I h